This notebook processes raw data downloaded from multiple research funding sources (e.g., UKRI, NIHR, EPMC, and the European Commission), harmonising and merging them into a unified and structured dataset suitable for analysis.

The code is functional more than it is elegant (i.e., it does the job, but it is not pretty!)

The notebook is structured into sequential sections: it begins by loading and inspecting the raw data files, proceeds through data cleaning and standardisation steps (including aligning column names, handling missing values, and creating consistent categorisations), and finally combines the processed datasets into a `Research projects.csv` and a `Research projects awardess.csv`.

Throughout the process I intercede analysis/exploration cells that helped me make decisions on which transformations to produce. I have left those in the script for transparency.



# Processing raw downloads into project and awardee dfs

In [23]:
import pandas as pd
import numpy as np
import re as re

## Europe PMC

Raw download from EPMC's API can be found in the link below from my google drive.

https://drive.google.com/file/d/1yNPFGdRP-jfz1wNtrLUqSCySIk3AumkF/view?usp=sharing

In [2]:
epmc_projects = pd.read_csv('/content/drive/MyDrive/Bulk grant data/EPMC/EPMC_projects.csv', low_memory=False)


Because EPMC provides all project details through a single download or API endpoint, the data contains multiple rows per project—one row for each participating institution.

However, EPMC does not specify which institution is the lead or contracting organisation. This presents a problem because the final project table we intend to create should have exactly one unique entry per `grant_id`, with only one institution associated.

Fortunately, for EPMC, 94.3% of grants have just one participating institution. For the remaining 5.7%, when duplicates are removed based on grant_id, only the first listed institution is retained in the project table. This approach is not ideal, since the retained institution may not necessarily be the lead organisation.

In [3]:
# 1. Count rows per grant_id
grant_counts = epmc_projects['grant_id'].value_counts()

# 2. How many grant_ids have more than 1 row
grants_with_multiple_rows = grant_counts[grant_counts > 1]
num_grants_multiple_rows = len(grants_with_multiple_rows)

print(f"Number of grant_ids with more than 1 row: {num_grants_multiple_rows}")

# 3. What % of total grants have more than one row
total_unique_grants = epmc_projects['grant_id'].nunique()
percentage_multiple_rows = (num_grants_multiple_rows / total_unique_grants) * 100

print(f"Percentage of grants with more than one row: {percentage_multiple_rows:.2f}%")

# 4. Number of unique grants per funder_name with more than 1 row
# First, get the grant_ids that have more than 1 row
grant_ids_multiple_rows = grants_with_multiple_rows.index

# Filter the dataframe to only include rows with grant_ids that have multiple rows
df_multiple_rows = epmc_projects[epmc_projects['grant_id'].isin(grant_ids_multiple_rows)]

# Count unique grants per funder_name (for grants with multiple rows)
unique_grants_per_funder = df_multiple_rows.groupby('funder_name')['grant_id'].nunique().sort_values(ascending=False)

print("\nNumber of unique grants per funder_name (with more than 1 row):")
print(unique_grants_per_funder)

# 5. Funder_name with the largest % of unique grant_ids (not repeated rows)
# Calculate total grants per funder
total_grants_per_funder = epmc_projects.groupby('funder_name')['grant_id'].nunique()

# Calculate grants with single row per funder
df_single_rows = epmc_projects[~epmc_projects['grant_id'].isin(grant_ids_multiple_rows)]
single_row_grants_per_funder = df_single_rows.groupby('funder_name')['grant_id'].nunique()

# Fill missing values with 0 (funders that have no single-row grants)
single_row_grants_per_funder = single_row_grants_per_funder.reindex(total_grants_per_funder.index, fill_value=0)

# Calculate percentage of unique (single-row) grants per funder
percentage_unique_per_funder = (single_row_grants_per_funder / total_grants_per_funder * 100).sort_values(ascending=False)


# Additional analysis: Summary statistics
print(f"\n--- Summary Statistics ---")
print(f"Total number of rows in dataset: {len(epmc_projects)}")
print(f"Total unique grant_ids: {total_unique_grants}")
print(f"Grant_ids with single row: {total_unique_grants - num_grants_multiple_rows}")
print(f"Grant_ids with multiple rows: {num_grants_multiple_rows}")


# Show distribution of row counts per grant_id
print(f"\nDistribution of row counts per grant_id:")
row_count_distribution = grant_counts.value_counts().sort_index()
for rows, count in row_count_distribution.items():
    print(f"  {rows} row(s): {count} grants ({count/total_unique_grants*100:.1f}%)")

Number of grant_ids with more than 1 row: 6709
Percentage of grants with more than one row: 5.65%

Number of unique grants per funder_name (with more than 1 row):
funder_name
Wellcome Trust                                 2641
Cancer Research UK                             1370
Medical Research Council                        918
Telethon Italy                                  615
UK Research and Innovation                      425
                                               ... 
University of California                          1
Vancouver Coastal Health Research Institute       1
University of Colorado                            1
Victoria State Government (Australia)             1
Yorkshire Cancer Research                         1
Name: grant_id, Length: 115, dtype: int64

--- Summary Statistics ---
Total number of rows in dataset: 131243
Total unique grant_ids: 118682
Grant_ids with single row: 111973
Grant_ids with multiple rows: 6709

Distribution of row counts per grant_id:
 

In [4]:
# EPMC has multiple rows per grant (one per institution)
# We keep only the first institution as lead org
# This affects ~5.7% of grants


# First ensure you have a copy, not a view
epmc_projects = epmc_projects.copy()

# Remove duplicates
epmc_projects = epmc_projects.drop_duplicates(subset=['grant_id'])

# Use .loc for all assignments
epmc_projects.loc[:, 'award_holder_name'] = epmc_projects['given_name'] + " " + epmc_projects['family_name']
epmc_projects.loc[:, 'institution_ror_id'] = 'https://ror.org/' + epmc_projects['institution_ror_id'].astype(str)

I want to create a `country` column in the EPMC dataset using the `grant_currency` field as a proxy indicator for country location (e.g., GBP → UK, USD → USA). Given EUR is used across multiple European countries, entries with EUR will collectively be labelled as 'Based in the EU'. Again there might be occasions where a funder funds in a certain currency, but the organisation is based elsewhere. We expect these to be a relatively small percentage of the total, but future versions of this project will analyse `institution_name` directly to create the `country` column.

Note that around 30% of entries in `grant_currency` are missing (NaN). However, upon reviewing the top funders associated with these missing values, it's clear that most are UK-based funders who have not input currency information. To address this, we'll manually map these key funders to their appropriate countries and currencies, substantially improving data completeness.

In [5]:
# Count of values including empty/NaN values
currency_counts = epmc_projects['grant_currency'].value_counts(dropna=False)
print("Value counts including empty values:")
print(currency_counts)

# Percentage distribution including empty values
currency_percentage = epmc_projects['grant_currency'].value_counts(dropna=False, normalize=True) * 100
print("\nPercentage distribution including empty values:")
print(currency_percentage)

missing_currency_df = epmc_projects[epmc_projects['grant_currency'].isna()]

# Get distribution of funder_name for these rows
funder_distribution = missing_currency_df['funder_name'].value_counts()
print("Funder distribution for rows with missing grant_currency:")
print(funder_distribution)

# Calculate percentage
funder_percentage = missing_currency_df['funder_name'].value_counts(normalize=True) * 100
print("\nPercentage distribution:")
print(funder_percentage)

Value counts including empty values:
grant_currency
GBP    56253
NaN    36229
EUR    14118
CHF     4675
USD     3842
CAD     1580
INR      560
BRL      352
ARS      187
JPY      128
AUD      124
SEK      124
DKK      116
NZD       96
NOK       55
PLN       53
PEN       37
COP       32
PYG       31
PAB       21
HUF       20
RWF       17
BGN       13
ZAR       10
MUR        6
CLP        3
SGD        1
Name: count, dtype: int64

Percentage distribution including empty values:
grant_currency
GBP    47.397690
NaN    30.525855
EUR    11.895554
CHF     3.939065
USD     3.237195
CAD     1.331277
INR     0.471845
BRL     0.296588
ARS     0.157563
JPY     0.107850
AUD     0.104480
SEK     0.104480
DKK     0.097739
NZD     0.080888
NOK     0.046342
PLN     0.044657
PEN     0.031175
COP     0.026963
PYG     0.026120
PAB     0.017694
HUF     0.016852
RWF     0.014324
BGN     0.010954
ZAR     0.008426
MUR     0.005055
CLP     0.002528
SGD     0.000843
Name: proportion, dtype: float64
Funder distribu

In [6]:
import numpy as np
epmc_projects['country'] = 'Unknown'  # Default value

# Apply the conditions
conditions = [
    (epmc_projects['grant_currency'].isna()),  # Missing values
    (epmc_projects['grant_currency'] == 'GBP'),
    (epmc_projects['grant_currency'] == 'EUR'),
    (epmc_projects['grant_currency'] == 'USD'),
    (epmc_projects['grant_currency'] == 'CHF'),
    (epmc_projects['grant_currency'] == 'INR'),
    (epmc_projects['grant_currency'] == 'CAD'),
    (epmc_projects['grant_currency'] == 'BRL'),
    (epmc_projects['grant_currency'] == 'ARS'),
    (epmc_projects['grant_currency'] == 'AUD'),
    (epmc_projects['grant_currency'] == 'JPY'),
    (epmc_projects['grant_currency'] == 'SEK'),
    (epmc_projects['grant_currency'] == 'DKK'),
    (epmc_projects['grant_currency'] == 'NZD'),
    (epmc_projects['grant_currency'] == 'NOK'),
    (epmc_projects['grant_currency'] == 'PLN'),
    (epmc_projects['grant_currency'] == 'PEN'),
    (epmc_projects['grant_currency'] == 'COP'),
    (epmc_projects['grant_currency'] == 'PYG'),
    (epmc_projects['grant_currency'] == 'PAB'),
    (epmc_projects['grant_currency'] == 'HUF'),
    (epmc_projects['grant_currency'] == 'RWF'),
    (epmc_projects['grant_currency'] == 'BGN'),
    (epmc_projects['grant_currency'] == 'ZAR'),
    (epmc_projects['grant_currency'] == 'SGD'),
    (epmc_projects['grant_currency'] == 'MUR'),
    (epmc_projects['grant_currency'] == 'CLP')
]

choices = [
    'Unknown',           # For missing values
    'GB',           # For GBP
    'Based in EU',  # For EUR
    'US',           # For USD
    'Switzerland',  # For CHF
    'India',        # For INR
    'Canada',       # For CAD
    'Brazil',       # For BRL
    'Argentina',    # For ARS
    'Australia',    # For AUD
    'Japan',        # For JPY
    'Sweden',       # For SEK
    'Denmark',      # For DKK
    'New Zealand',  # For NZD
    'Norway',       # For NOK
    'Poland',       # For PLN
    'Peru',         # For PEN
    'Colombia',     # For COP
    'Paraguay',     # For PYG
    'Panama',       # For PAB
    'Hungary',      # For HUF
    'Rwanda',       # For RWF
    'Bulgaria',     # For BGN
    'South Africa', # For ZAR
    'Singapore',    # For SGD
    'Mauritius',    # For MUR
    'Chile'         # For CLP
]

# Apply the conditions and choices to create the country column
epmc_projects['country'] = np.select(conditions, choices, default='Unknown')

# Create mapping of funder names to country codes and currencies
funder_mapping = {
    'Cancer Research UK': {'country': 'GB', 'currency': 'GBP'},
    'Biotechnology and Biological Sciences Research Council': {'country': 'GB', 'currency': 'GBP'},
    'Wellcome Trust': {'country': 'GB', 'currency': 'GBP'},
    'ZonMw': {'country': 'NL', 'currency': 'EUR'},
    'Medical Research Council': {'country': 'GB', 'currency': 'GBP'},
    'Austrian Science Fund FWF': {'country': 'AT', 'currency': 'EUR'},
    'Versus Arthritis': {'country': 'GB', 'currency': 'GBP'},
    'Swiss National Science Foundation': {'country': 'CH', 'currency': 'CHF'},
    'British Heart Foundation': {'country': 'GB', 'currency': 'GBP'},
    'World Health Organization': {'country': 'CH', 'currency': 'CHF'},  # Based in Switzerland
    'Blood Cancer UK': {'country': 'GB', 'currency': 'GBP'},
    'Chief Scientist Office': {'country': 'GB', 'currency': 'GBP'},  # UK (Scotland)
    'National Medical Research Council Singapore': {'country': 'SG', 'currency': 'SGD'},
    'UK Research and Innovation': {'country': 'GB', 'currency': 'GBP'},
    'Breast Cancer Now': {'country': 'GB', 'currency': 'GBP'}
}

# Apply the override for rows where country is "Unknown"
for funder_name, mapping in funder_mapping.items():
    mask = (epmc_projects['country'] == 'Unknown') & (epmc_projects['funder_name'] == funder_name)
    epmc_projects.loc[mask, 'country'] = mapping['country']
    epmc_projects.loc[mask, 'grant_currency'] = mapping['currency']



The final result is a df `epmc_projects`  where most of rows have country information, either through their `grant_currency`, or through their `funder_name` information.

Below we create a separate `epmc_awardees` df.

In [7]:
# Create a copy when selecting columns
epmc_awardees = epmc_projects[['grant_id', 'award_holder_name', 'orcid', 'institution_name']].copy()

# Use .loc for assignment
epmc_awardees.loc[:, 'orcid'] = 'orcid.org/' + epmc_awardees['orcid']

## UKRI Gateway to Research



Raw download from the different UKRI GTR endpoints can be found in the link below from my google drive

https://drive.google.com/drive/folders/1jQfJidvFSyTtHLNPn0RWqVPZWrk0aT6j?usp=sharing

In [9]:
projects = '/content/drive/MyDrive/Bulk grant data/UKRI Gateway/UKRI_projects.csv'
df_projects = pd.read_csv(projects)
funds = '/content/drive/MyDrive/Bulk grant data/UKRI Gateway/UKRI_funds.csv'
df_funds = pd.read_csv(funds)
persons = '/content/drive/MyDrive/Bulk grant data/UKRI Gateway/UKRI_persons.csv'
df_persons = pd.read_csv(persons)
orgs = '/content/drive/MyDrive/Bulk grant data/UKRI Gateway/UKRI_orgs.csv'
df_orgs = pd.read_csv(orgs)

We merge data from multiple CSV files downloaded from each UKRI GTR endpoint.

These files do not include a `country` column. However, they do provide a `region` column, which mostly contains UK regions but has some entries marked as "Unknown". We use this `region` information to create a new `country` column, assigning the value "GB" to all rows except those marked "Unknown".

In [10]:
projects_funds = df_projects.merge(
    df_funds,
    on='project_id',
    how='left'
)

projects_funds_orgs = projects_funds.merge(
    df_orgs,
    left_on='lead_org_id',
    right_on='org_id',
    how='left'
)

projects_funds_orgs['lead_org_id'] = 'https://gtr.ukri.org/organisation/' + projects_funds_orgs['lead_org_id'].astype(str)

# Add country column based on region
projects_funds_orgs['country'] = projects_funds_orgs['region'].apply(
    lambda x: 'Unknown' if x == 'Unknown' else 'GB'
)

ukri_projects = projects_funds_orgs

`ukri_projects` is initially structured in a wide format, containing separate columns for each potential participant role within a project. Each cell holds one or more researcher IDs, indicating participants fulfilling that specific role. If a column does not apply to specific grant (e.g., pi_super_id for entries that are not PhD grants_, the cell is empty.

We reshape ("melt") the dataset to a long format, consolidating all participant IDs into a single column. We then merge additional details from the `df_persons` DataFrame, using the researcher ID as the linking column.

In [11]:
# Create list of columns to unpivot
value_columns = [
    'coi_person_ids',
    'fellow_person_ids',
    'pi_person_ids',
    'student_person_ids',
    'super_person_ids'
]

# First get the base dataset with project info
projects_funds_orgs_melted = pd.melt(
    projects_funds_orgs,
    id_vars=['project_id'],
    value_vars=value_columns,
    var_name='role_type',
    value_name='person_id'
)

# Clean it up
projects_funds_orgs_melted = projects_funds_orgs_melted.dropna(subset=['person_id'])
projects_funds_orgs_melted = (
    projects_funds_orgs_melted.assign(
        person_id=projects_funds_orgs_melted['person_id'].str.split(';')
    )
    .explode('person_id'))
projects_funds_orgs_melted['person_id'] = projects_funds_orgs_melted['person_id'].str.strip()
projects_funds_orgs_melted = projects_funds_orgs_melted.drop_duplicates(
    subset=['project_id', 'person_id'])



# merge - combining with persons data
persons_merge = projects_funds_orgs_melted.merge(
    df_persons[['person_id', 'full_name', 'organisation_id']],
    on='person_id',
    how='left'
).merge(
    df_orgs[['org_id', 'name', 'region']],
    left_on='organisation_id',
    right_on='org_id',
    how='left'
)

# Step 2: Print info to verify the first merge worked
print("Shape after first merge:", persons_merge.shape)
print("Sample of merged data:")
print(persons_merge.head())

# Step 3: Second merge - adding project data
# Let's explicitly select both columns we're using from df_projects
projects_subset = df_projects[['project_id', 'grant_id']]

# Step 4: Final merge
final_merge = persons_merge.merge(
    projects_subset,
    on='project_id',
    how='left'
)

final_merge['person_id'] = 'https://gtr.ukri.org/person/' + final_merge['person_id']
ukri_awardees = final_merge

Shape after first merge: (287009, 8)
Sample of merged data:
                             project_id       role_type  \
0  037CCCC0-3B1F-4364-BFE8-021B1C59F637  coi_person_ids   
1  03AB4796-F201-4B16-9911-02272D373118  coi_person_ids   
2  03AB4796-F201-4B16-9911-02272D373118  coi_person_ids   
3  07828D21-A20E-4368-9D88-015A30686E6F  coi_person_ids   
4  007662E8-5AFF-497A-A1FD-00246571593F  coi_person_ids   

                              person_id           full_name  \
0  245BBF35-3CEE-46D2-86E9-7806E57A17E0          Mercy Ette   
1  9E90CB36-3ED6-4099-9170-C7BD7F82F8F6  Nishan Canagarajah   
2  938775B7-C1DF-4EC7-94E8-FC52C55F6DDB          David Bull   
3  3B56736B-6873-42E8-8CB9-30A348B1BECD        Colin Butter   
4  58DB3AC2-B583-47EC-91E9-A6CE5D9D3E0E        James Murray   

                        organisation_id                                org_id  \
0  EAAE50F2-29A1-48A5-92E9-BF2C930203E5  EAAE50F2-29A1-48A5-92E9-BF2C930203E5   
1  F9431CE1-F5FE-4418-A853-61549E962D88  F94

## NIHR Funding and Awards

Bulk NIHR download available here

https://drive.google.com/file/d/1fZZnYV4xciFlKHxJAWY0xHvihQ1XG_U7/view?usp=sharing

In [12]:
nihr = '/content/drive/MyDrive/Bulk grant data/NIHR/NIHR.csv'
nihr_projects = pd.read_csv(nihr)

NIHR includes a `institutioncountry` column, but the data quality is very poor—most entries contain irrelevant values, and a few rows list multiple countries.

However, NIHR provides accurate `latitude` and `longitude` for the lead organisations. We leverage these coordinates, using predefined latitude and longitude boundaries for the UK, to determine whether each grant is funded within the UK or abroad.

In [14]:
import pandas as pd
import numpy as np

# Define UK coordinate boundaries
# UK roughly spans:
# Latitude: 49.5° to 61° North
# Longitude: -8.5° to 2° East (negative values are West)
UK_LAT_MIN = 49.5
UK_LAT_MAX = 61.0
UK_LON_MIN = -8.5
UK_LON_MAX = 2.0

# Create country_final column
def assign_country(row):
    # Check if coordinates are available
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        # Check if coordinates fall within UK bounds
        if (UK_LAT_MIN <= row['latitude'] <= UK_LAT_MAX and
            UK_LON_MIN <= row['longitude'] <= UK_LON_MAX):
            return 'GB'
        else:
            return row['institutioncountry']
    else:
        # If coordinates are missing, use institutioncountry
        return row['institutioncountry']

# Apply the function to create the new column
nihr_projects['country_final'] = nihr_projects.apply(assign_country, axis=1)

# Verify the results
print("Country assignment summary:")
print(nihr_projects['country_final'].value_counts())

# Check how many were assigned GB based on coordinates vs institutioncountry
gb_from_coords = ((nihr_projects['latitude'].between(UK_LAT_MIN, UK_LAT_MAX)) &
                  (nihr_projects['longitude'].between(UK_LON_MIN, UK_LON_MAX)) &
                  (nihr_projects['country_final'] == 'GB')).sum()

gb_from_institution = (nihr_projects['country_final'] == 'GB').sum() - gb_from_coords

print(f"\nGB assignments:")
print(f"From coordinates: {gb_from_coords}")
print(f"From institutioncountry: {gb_from_institution}")

Country assignment summary:
country_final
GB                                                                                                                                                              10080
Award does not have an ODA Downstream Partner                                                                                                                      53
India/Peru/Nepal/Mozambique                                                                                                                                         1
India/Brazil/India/South Africa                                                                                                                                     1
Kenya/UK institution/Kenya                                                                                                                                          1
Bangladesh/UK institution/Kenya/Kenya                                                                                           

Below, I extract the relevant information for awardees from multiple columns, each of which stores details for multiple awardees within single cells.

To construct an awardee-level table, I first separate the individual awardees' details from each of these columns, and then recombine this information at the awardee level, making sure that each row corresponds to an individual awardee.

In [15]:
# Create an explicit copy of the DataFrame
nihr_awardee_table = nihr_projects[[
    'project_id',
    "award_holder_name",
    "orcid",
    "involvement_type",
    'contracted_organisation',
    'organisation_type'
]].copy()

# Clean up the awardee names
nihr_awardee_table["award_holder_name"] = nihr_awardee_table["award_holder_name"].str.replace(
    r'\b(Professor|Dr|Associate|Mr|Ms|Miss|Mrs)\b', '', regex=True
).str.strip()


# Ensure the relevant columns are strings and handle NaN
for col in ["award_holder_name", "orcid", "involvement_type", "contracted_organisation", "organisation_type"]:
    nihr_awardee_table[col] = nihr_awardee_table[col].fillna("").astype(str)

def create_row_df(row):
    try:
        # Split all fields
        holders = row["award_holder_name"].split("/") if "/" in row["award_holder_name"] else [row["award_holder_name"]]
        ids = row["orcid"].split("/") if "/" in row["orcid"] else [row["orcid"]]
        roles = row["involvement_type"].split("/") if "/" in row["involvement_type"] else [row["involvement_type"]]
        orgs = row["contracted_organisation"].split("/") if "/" in row["contracted_organisation"] else [row["contracted_organisation"]]
        org_types = row["organisation_type"].split("/") if "/" in row["organisation_type"] else [row["organisation_type"]]

        # Get the number of award holders as our base length
        num_holders = len(holders)

        # If we have a single organization/org type, repeat it for all holders
        if len(orgs) == 1:
            orgs = orgs * num_holders
        if len(org_types) == 1:
            org_types = org_types * num_holders

        # Pad roles and ids if needed
        if len(roles) < num_holders:
            roles = roles + [''] * (num_holders - len(roles))
        if len(ids) < num_holders:
            ids = ids + [''] * (num_holders - len(ids))

        # Make sure all lists are exactly the same length as holders
        orgs = orgs[:num_holders]
        org_types = org_types[:num_holders]
        roles = roles[:num_holders]
        ids = ids[:num_holders]

        # Create lists for Project_ID
        project_ids = [row["project_id"]] * num_holders

        # Final length check with detailed debugging
        lengths = [len(project_ids), len(holders), len(ids), len(roles), len(orgs), len(org_types)]
        if len(set(lengths)) != 1:
            print(f"\nLength mismatch for Project ID {row['project_id']}:")
            print(f"project_ids ({len(project_ids)}): {project_ids}")
            print(f"holders ({len(holders)}): {holders}")
            print(f"ids ({len(ids)}): {ids}")
            print(f"roles ({len(roles)}): {roles}")
            print(f"orgs ({len(orgs)}): {orgs}")
            print(f"org_types ({len(org_types)}): {org_types}")
            print(f"All lengths: {lengths}")
            # Skip this row to continue processing
            return pd.DataFrame()

        # Create the DataFrame
        df_data = {
            "Project_ID": project_ids,
            "Award Holder": holders,
            "orcid": ids,
            "Role": roles,
            "Organisation": orgs,
            "Organisation Type": org_types
        }

        # Double-check before creating DataFrame
        for key, value in df_data.items():
            if len(value) != num_holders:
                print(f"\nFinal check failed for Project ID {row['project_id']}:")
                print(f"Column '{key}' has length {len(value)} but expected {num_holders}")
                print(f"Value: {value}")
                return pd.DataFrame()

        return pd.DataFrame(df_data)

    except Exception as e:
        print(f"\nUnexpected error processing Project ID {row['project_id']}:")
        print(f"Error: {str(e)}")
        print(f"Row data: {dict(row)}")
        # Return empty DataFrame to continue processing
        return pd.DataFrame()

# Let's first inspect a few rows before transformation
print("Sample of original data:")
print(nihr_awardee_table.head())

# Check for rows with "Unknown" organization
unknown_org_rows = nihr_awardee_table[nihr_awardee_table['contracted_organisation'].str.contains('Unknown|Not Known', case=False, na=False)]
print(f"\nRows with Unknown/Not Known organizations: {len(unknown_org_rows)}")
if len(unknown_org_rows) > 0:
    print("Sample:")
    print(unknown_org_rows[['project_id', 'contracted_organisation', 'organisation_type']].head())
    print(f"Organization types for these rows: {unknown_org_rows['organisation_type'].unique()}")

# Process each row individually to catch problematic ones
print(f"\nProcessing {len(nihr_awardee_table)} rows...")
long_awardee_rows = []
for idx, row in nihr_awardee_table.iterrows():
    if idx % 1000 == 0:
        print(f"Processing row {idx}...")
    result_df = create_row_df(row)
    if not result_df.empty:
        long_awardee_rows.append(result_df)

# Concatenate all the individual DataFrames
if long_awardee_rows:
    long_awardee_table = pd.concat(long_awardee_rows, ignore_index=True)

    # Clean up whitespace in all relevant columns
    columns_to_strip = ["Award Holder", "Role", "Organisation", "Organisation Type"]
    for col in columns_to_strip:
        long_awardee_table[col] = long_awardee_table[col].str.strip()

    # Remove rows where Award Holder is empty
    long_awardee_table = long_awardee_table[long_awardee_table["Award Holder"] != ""]

    # Add ORCID prefix only to non-empty ORCID values, excluding "Not Provided"
    long_awardee_table['orcid'] = long_awardee_table['orcid'].apply(
        lambda x: 'orcid.org/' + x if x and x.strip() and x.strip().lower() != 'not provided' else ''
    )

    print(f"\nSuccessfully processed data. Final table has {len(long_awardee_table)} rows.")
else:
    print("\nNo valid rows were processed!")
    long_awardee_table = pd.DataFrame()


nihr_awardees = long_awardee_table

Sample of original data:
  project_id  award_holder_name                orcid    involvement_type  \
0   001/0009        Ronan Lyons  0000-0001-5225-000X  Chief Investigator   
1   001/0012       James Thomas  0000-0003-4805-4190  Chief Investigator   
2   001/0020  Christine McGuire         Not Provided  Chief Investigator   
3   002/0025       Brian Miller         Not Provided  Chief Investigator   
4   002/0027       Roy Harrison         Not Provided  Chief Investigator   

                        contracted_organisation   organisation_type  
0                            Swansea University            Academic  
1  Institute of Education, University of London            Academic  
2                     University College London            Academic  
3            Institute of Occupational Medicine  Non-Profit/Charity  
4                      University of Birmingham            Academic  

Rows with Unknown/Not Known organizations: 7
Sample:
            project_id contracted_organisati

## European Commission's H2020 and Horizon programmes

Zip files are available for download from the EC website.

Information is scattered across several csvs.

Here I am linking to the relevant project and organisation merged data for both H2020 and Horizon:

https://drive.google.com/drive/folders/1F5ZPMQA8JMiQunuV6ZyZreYpCQe9R8pI?usp=sharing

In [16]:
ec_projects = pd.read_csv('/content/drive/MyDrive/Bulk grant data/European Commission/to share/EC full projects.csv')
ec_awardees = pd.read_csv('/content/drive/MyDrive/Bulk grant data/European Commission/to share/EC full organizations.csv', low_memory=False)

The key issue with EC grants is that most entries lack information on individual awardees (I was only able to find separate awardee details for Early Career Researchers funded through the Horizon 2020 programme, covering around 3% of entries).

Each EC grant lists multiple participating organisations—many of which do not receive funding at all. For the project table, I extract information only for the organisation labelled as the "coordinator". A small proportion of projects list more than one coordinator; in these cases, similarly to the EPMC dataset, I select only the first organisation listed.

In [17]:
# Filter ec_awardees for coordinators only
coordinators = ec_awardees[ec_awardees['role'] == 'coordinator']

# Merge with ec_projects
ec_projects = ec_projects.merge(
    coordinators[['projectID', 'name', 'country', 'organizationURL']],
    left_on='id',
    right_on='projectID',
    how='left'
).drop_duplicates(subset='id')

In [18]:
# First ensure you have a copy if ec_awardees was created by subsetting
ec_awardees = ec_awardees.copy()  # Add this if ec_awardees was created from another DataFrame

# Use .loc for assignment
ec_awardees.loc[:, 'award_holder_name'] = ec_awardees['firstName'] + ' ' + ec_awardees['lastName']

# Harmonising columns and combinings datasets

In this section I am going to select the minimum set of common columns across the dataset. I need to rename each column from the original nomenclature to a final standardised column name.

I also need to be careful with duplicates: EPMC contains information from UKRI Research Councils, but I cannot simply delete all their rows, because they also have information on infrastructure grants that are not available through UKRI GTR. We rely on grant_id for that deduplication process.




## Projects

In [28]:
# Function to clean grant_ids
def clean_grant_id(series):
    # Convert to string, handling NaN values
    cleaned = series.astype(str)
    # Replace 'nan' strings (from converted NaN values) with actual NaN
    cleaned = cleaned.replace('nan', pd.NA)
    # Strip whitespace
    cleaned = cleaned.str.strip()
    return cleaned

# Apply to all datasets before creating the common dataframes
epmc_projects['grant_id'] = clean_grant_id(epmc_projects['grant_id'])
nihr_projects['project_id'] = clean_grant_id(nihr_projects['project_id'])
ukri_projects['grant_id'] = clean_grant_id(ukri_projects['grant_id'])
ec_projects['id'] = clean_grant_id(ec_projects['id'])



# Harmonise columns across datasets
# EPMC
epmc_common = epmc_projects.rename(columns={
    'grant_id': 'grant_id',
    'grant_title': 'title',
    'scientific_abstract': 'abstract',
 #   'lay_abstract': 'lay_abstract',
    'grant_type': 'grant_category',
 #   'grant_stream': 'grant_stream',
    'funder_name': 'funder',
    'institution_name': 'lead_org_name',
    'institution_ror_id': 'lead_org_id',
   'country': 'country',
    'start_date': 'start_date',
    'end_date': 'end_date',
    'grant_amount': 'grant_amount',
    'grant_currency': 'grant_currency'
}).assign(
    source='Europe PMC')[['grant_id', 'source', 'title', 'abstract', 'grant_category', 'funder', 'lead_org_name','lead_org_id', 'country', 'start_date', 'end_date', 'grant_amount', 'grant_currency']]

# NIHR

nihr_common = nihr_projects.rename(columns={
    'project_id': 'grant_id',
   # 'funding_and_awards_link': 'grant_url',
    'project_title': 'title',
    'scientific_abstract': 'abstract',
  #  'plain_english_abstract': 'lay_abstract',
    'programme_type': 'grant_category',
 #   'programme': 'grant_stream',
    'funder': 'funder',
    'contracted_organisation': 'lead_org_name',
    'country_final': 'country',
    'start_date': 'start_date',
    'end_date': 'end_date',
    'award_amount_from_dh': 'grant_amount'
}).assign(
    source='NIHR Funding and Awards',
    **{'grant_currency': 'GBP'})[['grant_id', 'source', 'title', 'abstract', 'grant_category', 'funder', 'lead_org_name', 'country', 'start_date', 'end_date', 'grant_amount', 'grant_currency']]

# UKRI
ukri_common = ukri_projects.rename(columns={
    'grant_id': 'grant_id',
    'title': 'title',
    'abstract': 'abstract',
    'grant_category': 'grant_category',
    'lead_funder': 'funder',
    'name': 'lead_org_name',
    'lead_org_id': 'lead_org_id',
    'country': 'country',
    'start_date': 'start_date',
    'end_date': 'end_date',
    'amount': 'grant_amount',
    'currency': 'grant_currency'
}).assign(
    source = 'UKRI Gateway to Research')[['grant_id', 'source', 'title', 'abstract', 'grant_category', 'funder', 'lead_org_name','lead_org_id','country', 'start_date', 'end_date', 'grant_amount', 'grant_currency']]


# Rename existing columns and create new ones in one step
ec_common = ec_projects.rename(columns={
    'id': 'grant_id',
    'title': 'title',
    'objective': 'abstract',
    'frameworkProgramme': 'grant_category',
    'fundingScheme': 'grant_stream',
    'name': 'lead_org_name',
    'country': 'country',
    'organizationURL': 'lead_org_id',
    'startDate': 'start_date',
    'endDate': 'end_date',
    'totalCost': 'grant_amount'
}).assign(
    funder='European Commission',
    **{'grant_currency': 'EUR'},
    **{'source': 'European Commission'}
)[['grant_id', 'source', 'title', 'abstract', 'grant_category', 'funder', 'lead_org_name','lead_org_id', 'country',  'start_date', 'end_date', 'grant_amount', 'grant_currency']]

# First, combine UKRI and NIHR datasets
priority_df = pd.concat([ukri_common, nihr_common], ignore_index=True)

# Get list of IDs from UKRI and NIHR
existing_ids = priority_df['grant_id'].unique()

# Filter EPMC to only include rows where the ID isn't in UKRI or NIHR
epmc_unique = epmc_common[~epmc_common['grant_id'].isin(existing_ids)]
epmc_unique = epmc_unique.loc[:, ~epmc_unique.columns.duplicated()]

# Now combine all datasets
final_combined_df = pd.concat([priority_df, epmc_unique, ec_common], ignore_index=True)
final_combined_df = final_combined_df.dropna(subset=['grant_id'])


We are also going to standardise funder and organisation names, since there were discrepancies in nomenclature across sources. This manual mapping is not exhaustive but visual inspection of the top funders and orgs shows that is effective at fixing the evident issues.

In [29]:
# Create a mapping dictionary for funder standardization
funder_mapping = {
    # UK Research Councils
    'EPSRC': 'Engineering and Physical Sciences Research Council',
    'BBSRC': 'Biotechnology and Biological Sciences Research Council',
    'ESRC': 'Economic and Social Research Council',
    'MRC': 'Medical Research Council',
    'NERC': 'Natural Environment Research Council',
    'AHRC': 'Arts and Humanities Research Council',
    'STFC': 'Science and Technology Facilities Council',
    'NIHR (non-ODA)': 'National Institute for Health Research',
    'National Institute for Health Research (NIHR)': 'National Institute for Health Research',

    # UK Funding Bodies
    'UKRI': 'UK Research and Innovation',
    'UKRI FLF': 'UK Research and Innovation Future Leaders Fellowship',
    'UK Research and Innovation': 'UK Research and Innovation',
    'ISCF': 'Industrial Strategy Challenge Fund',
    'GCRF': 'Global Challenges Research Fund',
    'SPF': 'Strategic Priorities Fund',

    # International Organizations
    'European Research Council': 'European Research Council',
    'Dutch Research Council (NWO)': 'Dutch Research Council',
    'ZonMw': 'Netherlands Organisation for Health Research and Development',
    'NC3Rs': 'National Centre for the Replacement, Refinement and Reduction of Animals in Research',
    'RNID': 'Royal National Institute for Deaf People',
    'DBT/Wellcome Trust India Alliance': 'Department of Biotechnology/Wellcome Trust India Alliance',
    'COVID': 'COVID-19 Research Funding'
    # Add any other mappings based on your full dataset
}

# Apply the mapping to standardize funder names
final_combined_df['funder_standardised'] = final_combined_df['funder'].map(funder_mapping).fillna(final_combined_df['funder'])

def standardize_org_name(name):
    # Check if the value is NaN or float
    if pd.isna(name) or not isinstance(name, str):
        return "Unknown"  # You can change this to another default value

    # First convert to title case
    name = name.title()

    # Fix lowercase words (articles, conjunctions, prepositions)
    lowercase_words = ['of', 'and', 'in', 'for', 'de', 'la']  # Removed 'the' from this list
    for word in lowercase_words:
        # Use word boundary to ensure we match whole words
        name = re.sub(r'\b' + word.title() + r'\b', word, name, flags=re.IGNORECASE)

    # Now only lowercase 'the' when it's not at the beginning of the name
    name = re.sub(r'(?<!^)(\s)The\s', r'\1the ', name)

    # Fix specific institution names
    if "King'S College London" in name:
        name = name.replace("King'S College London", "King's College London")
    if "Queen'S" in name:
        name = name.replace("Queen'S", "Queen's")
    if "University Of Manchester" in name or "university of manchester" in name.lower():
        name = "The University of Manchester"
    if "London Sch" in name or "london sch of hygiene" in name.lower():
        name = "London School of Hygiene & Tropical Medicine"
    if "Ucl" in name:
        name = name.replace("Ucl", "UCL")
    if "University College London" in name:
        name = "University College London"
    if "Centre National De La Recherche Scientifique" in name:
        name = "Centre National de la Recherche Scientifique (CNRS)"
    if "Mrc" in name:
        name = name.replace("Mrc", "MRC")

    # Fix abbreviations that should remain uppercase
    abbreviations = ['ucl', 'cnrs', 'mrc']
    for abbr in abbreviations:
        name = re.sub(r'\b' + abbr + r'\b', abbr.upper(), name, flags=re.IGNORECASE)

    # Handle special cases
    name = name.replace("Univ Of", "University of")
    name = name.replace("Univ ", "University ")
    name = name.replace(", Univ", ", University")

    return name

# Apply the standardization to create a new column
final_combined_df['lead_org_name_standardised'] = final_combined_df['lead_org_name'].apply(standardize_org_name)

In [31]:
# Analyze column quality (% empty/missing values)
def analyze_column_quality(df):
    """Analyze the percentage of missing values in each column"""

    total_rows = len(df)
    quality_stats = []

    for column in df.columns:
        # Count different types of missing values
        null_count = df[column].isnull().sum()
        empty_string_count = (df[column] == '').sum() if df[column].dtype == 'object' else 0
        total_missing = null_count + empty_string_count

        # Calculate percentages
        missing_pct = (total_missing / total_rows) * 100
        complete_pct = 100 - missing_pct

        quality_stats.append({
            'column': column,
            'total_rows': total_rows,
            'missing_count': total_missing,
            'missing_percentage': round(missing_pct, 2),
            'complete_percentage': round(complete_pct, 2),
            'data_type': str(df[column].dtype)
        })

    # Create summary DataFrame
    quality_df = pd.DataFrame(quality_stats)

    # Sort by missing percentage (highest first)
    quality_df = quality_df.sort_values('missing_percentage', ascending=False)

    return quality_df

# Run the analysis
column_quality = analyze_column_quality(final_combined_df)
print("Column Quality Analysis:")
print("=" * 80)
print(column_quality.to_string(index=False))

# Summary statistics
print("\n" + "=" * 80)
print("SUMMARY:")
print(f"Total rows in dataset: {len(final_combined_df):,}")
print(f"Total columns: {len(final_combined_df.columns)}")
print(f"Columns with >50% missing: {len(column_quality[column_quality['missing_percentage'] > 50])}")
print(f"Columns with >25% missing: {len(column_quality[column_quality['missing_percentage'] > 25])}")
print(f"Columns with complete data: {len(column_quality[column_quality['missing_percentage'] == 0])}")

Column Quality Analysis:
                    column  total_rows  missing_count  missing_percentage  complete_percentage data_type
              grant_amount      322422          78804               24.44                75.56    object
                  end_date      322422          67211               20.85                79.15    object
                start_date      322422          63803               19.79                80.21    object
            grant_currency      322422          49077               15.22                84.78    object
                  abstract      322422          39772               12.34                87.66    object
               lead_org_id      322422          20852                6.47                93.53    object
                     title      322422          19976                6.20                93.80    object
            grant_category      322422          11591                3.59                96.41    object
             lead_org_name    

In [32]:
# If you want to compare quality across data sources
# Add source labels before concatenating (in your original code):

# When creating the dataframes, add a source column:
# priority_df['source'] = 'UKRI_NIHR'
# epmc_unique['source'] = 'EPMC'
# ec_common['source'] = 'EC'

# Then analyze by source:
def analyze_by_source(df):
    """Analyze column quality grouped by data source"""
    if 'source' in df.columns:
        quality_by_source = df.groupby('source').apply(
            lambda x: pd.Series({
                col: f"{(x[col].isnull().sum() / len(x)) * 100:.1f}%"
                for col in x.columns if col != 'source'
            })
        )
        return quality_by_source
    else:
        print("No 'source' column found. Add source labels to track data origin.")
        return None

analyze_by_source(final_combined_df)

<ipython-input-32-b705099e43b6>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  quality_by_source = df.groupby('source').apply(


,grant_id,title,abstract,grant_category,funder,lead_org_name,lead_org_id,country,start_date,end_date,grant_amount,grant_currency,funder_standardised,lead_org_name_standardised
source,,,,,,,,,,,,,,
Europe PMC,0.0%,21.2%,42.1%,12.3%,0.0%,10.8%,0.0%,0.0%,21.8%,25.4%,37.7%,6.2%,0.0%,0.0%
European Commission,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,20.9%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
NIHR Funding and Awards,0.0%,0.0%,0.3%,0.0%,0.0%,0.0%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
UKRI Gateway to Research,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,25.9%,25.9%,25.9%,25.9%,0.0%,0.0%


In [33]:
final_combined_df.to_csv('/content/drive/MyDrive/Bulk grant data/Research projects.csv', index=False)

In [36]:
# Make sure start_date is in datetime format
final_combined_df['start_date'] = pd.to_datetime(final_combined_df['start_date'], errors='coerce')

# First, find the top 10 funders by count of grant_ids
top_funders = final_combined_df.groupby('funder_standardised')['grant_id'].nunique().sort_values(ascending=False).head(10).index.tolist()

# Now, filter the dataframe to include only these top 10 funders
top_funders_df = final_combined_df[final_combined_df['funder_standardised'].isin(top_funders)]

# For each of these funders, find the earliest start_date
earliest_dates = top_funders_df.groupby('funder_standardised')['start_date'].min().reset_index()

# Sort by funder name for readability (optional)
earliest_dates = earliest_dates.sort_values('funder_standardised')

# If you want to sort by the count of grants instead, to maintain the top 10 order:
funder_counts = final_combined_df.groupby('funder_standardised')['grant_id'].nunique().reset_index()
funder_counts = funder_counts.rename(columns={'grant_id': 'grant_count'})
earliest_dates = earliest_dates.merge(funder_counts, on='funder_standardised')
earliest_dates = earliest_dates.sort_values('grant_count', ascending=False)

# Display the results
print("Earliest start dates for top 10 funders by grant count:")
print(earliest_dates[['funder_standardised', 'start_date', 'grant_count']])

Earliest start dates for top 10 funders by grant count:
                                 funder_standardised start_date  grant_count
4                                European Commission 2014-01-01        51311
3  Engineering and Physical Sciences Research Cou... 2006-01-01        40615
5                                        Innovate UK 2006-01-01        31593
1  Biotechnology and Biological Sciences Research... 1997-07-01        28751
9                                     Wellcome Trust 1984-12-01        18869
6                           Medical Research Council 1973-01-01        18520
2               Economic and Social Research Council 2006-01-01        14100
8               Natural Environment Research Council 2006-01-01        13209
7             National Institute for Health Research 1993-01-01        12032
0               Arts and Humanities Research Council 2006-01-01        11970


In [38]:
funder_counts

,funder_standardised,grant_count
0,ANR/Other Funders (France),16
1,ARAMIS,13
2,AXA France,10
3,Academy of Finland,52
4,African Academy of Sciences,12
...,...,...
281,Worldwide Cancer Research,810
282,Yale University,21
283,York University,10
284,Yorkshire Cancer Research,192


## Awardees

In [39]:
# Function to standardize data format
def standardize_data(df, source):
    if source == 'NIHR Funding and Awards':
        standardized = pd.DataFrame({
            'researcher_name': df['Award Holder'],
            'researcher_id': df['orcid'],
            'grant_id': df['Project_ID'],
            'source': source,
            'role': df['Role'],
            'organisation': df['Organisation'],
    #        'country': df['country']

        })

    elif source == 'UKRI Gateway to Research':
        standardized = pd.DataFrame({
            'researcher_name': df['full_name'],
            'researcher_id': df['person_id'],
            'grant_id': df['grant_id'],
            'source': source,
            'role': df['role_type'],
            'organisation': df['name'],
     #       'country': df['country']
        })

    elif source == 'Europe PMC':
        standardized = pd.DataFrame({
            'researcher_name': df['award_holder_name'],
            'researcher_id': df['orcid'],
            'grant_id': df['grant_id'],
            'source': source,
            'role': "EPMC Awardee",
            'organisation': df['institution_name'],
      #      'country': df['country']
        })

    elif source == 'European Commission':
        standardized = pd.DataFrame({
            'researcher_name': df['award_holder_name'],
         #   'researcher_id': df['orcid'],
            'grant_id': df['projectID'],
            'source': source,
            'role': df['role'],
            'organisation': df['name'],
       #     'country': df['country']
        })

    return standardized

# Standardize each dataset
nihr_standardized = standardize_data(nihr_awardees, 'NIHR Funding and Awards')
ukri_standardized = standardize_data(ukri_awardees, 'UKRI Gateway to Research')
epmc_standardized = standardize_data(epmc_awardees, 'Europe PMC')
ec_standardized = standardize_data(ec_awardees, 'European Commission')

# First, combine only UKRI, NIHR, and EPMC for deduplication
# (excluding EC because most researcher names are empty)
combined_for_dedup = pd.concat([
    ukri_standardized,    # Highest priority
    nihr_standardized,    # Second priority
    epmc_standardized     # Third priority
], ignore_index=True)

# Print statistics before deduplication
print("\nDataset sizes before deduplication:")
print(f"NIHR entries: {len(nihr_standardized)}")
print(f"UKRI entries: {len(ukri_standardized)}")
print(f"EPMC entries: {len(epmc_standardized)}")
print(f"EC entries: {len(ec_standardized)} (excluded from researcher name deduplication)")
print(f"Combined dataset size (for deduplication): {len(combined_for_dedup)}")

# Remove duplicates based on researcher_name and grant_id
# Keep first occurrence (highest priority source)
# Only applies to UKRI, NIHR, and EPMC data
deduplicated_subset = combined_for_dedup.drop_duplicates(
    subset=['researcher_name', 'grant_id'],
    keep='first'
)

# Now add EC data back to the deduplicated dataset
combined_authors_dedup = pd.concat([
    deduplicated_subset,
    ec_standardized
], ignore_index=True)

# Remove rows where both researcher_name and organisation are empty/null
combined_authors_dedup = combined_authors_dedup[
    ~(combined_authors_dedup['researcher_name'].isna() &
      combined_authors_dedup['organisation'].isna())
]

# Print statistics after deduplication
print("\nDataset sizes after deduplication:")
print(f"Deduplicated subset (UKRI/NIHR/EPMC): {len(deduplicated_subset)}")
print(f"Duplicates removed from UKRI/NIHR/EPMC: {len(combined_for_dedup) - len(deduplicated_subset)}")
print(f"Total combined dataset size (including EC): {len(combined_authors_dedup)}")



# Optional: Save the deduplicated dataset
# combined_authors_dedup.to_csv('combined_authors_deduplicated.csv', index=False)



Dataset sizes before deduplication:
NIHR entries: 11408
UKRI entries: 287009
EPMC entries: 118683
EC entries: 279048 (excluded from researcher name deduplication)
Combined dataset size (for deduplication): 417100

Dataset sizes after deduplication:
Deduplicated subset (UKRI/NIHR/EPMC): 400799
Duplicates removed from UKRI/NIHR/EPMC: 16301
Total combined dataset size (including EC): 676256


In [40]:
import re
import pandas as pd

def standardize_org_name(name):
    # Check if the value is NaN or float
    if pd.isna(name) or not isinstance(name, str):
        return "Unknown"  # You can change this to another default value

    # First convert to title case
    name = name.title()

    # Fix lowercase words (articles, conjunctions, prepositions)
    lowercase_words = ['of', 'and', 'in', 'for', 'de', 'la']  # Removed 'the' from this list
    for word in lowercase_words:
        # Use word boundary to ensure we match whole words
        name = re.sub(r'\b' + word.title() + r'\b', word, name, flags=re.IGNORECASE)

    # Now only lowercase 'the' when it's not at the beginning of the name
    name = re.sub(r'(?<!^)(\s)The\s', r'\1the ', name)

    # Fix specific institution names
    if "King'S College London" in name:
        name = name.replace("King'S College London", "King's College London")
    if "Queen'S" in name:
        name = name.replace("Queen'S", "Queen's")
    if "University Of Manchester" in name or "university of manchester" in name.lower():
        name = "The University of Manchester"
    if "London Sch" in name or "london sch of hygiene" in name.lower():
        name = "London School of Hygiene & Tropical Medicine"
    if "Ucl" in name:
        name = name.replace("Ucl", "UCL")
    if "University College London" in name:
        name = "University College London"
    if "Centre National De La Recherche Scientifique" in name:
        name = "Centre National de la Recherche Scientifique (CNRS)"
    if "Mrc" in name:
        name = name.replace("Mrc", "MRC")

    # Fix abbreviations that should remain uppercase
    abbreviations = ['ucl', 'cnrs', 'mrc']
    for abbr in abbreviations:
        name = re.sub(r'\b' + abbr + r'\b', abbr.upper(), name, flags=re.IGNORECASE)

    # Handle special cases
    name = name.replace("Univ Of", "University of")
    name = name.replace("Univ ", "University ")
    name = name.replace(", Univ", ", University")

    return name

# Apply the standardization to create a new column
combined_authors_dedup['organisation_standardised'] = combined_authors_dedup['organisation'].apply(standardize_org_name)

In [42]:
# Then analyze by source:
def analyze_by_source(df):
    """Analyze column quality grouped by data source"""
    if 'source' in df.columns:
        quality_by_source = df.groupby('source').apply(
            lambda x: pd.Series({
                col: f"{(x[col].isnull().sum() / len(x)) * 100:.1f}%"
                for col in x.columns if col != 'source'
            })
        )
        return quality_by_source
    else:
        print("No 'source' column found. Add source labels to track data origin.")
        return None

analyze_by_source(combined_authors_dedup)

<ipython-input-42-a32c9372adc6>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  quality_by_source = df.groupby('source').apply(


,researcher_name,researcher_id,grant_id,role,organisation,organisation_standardised
source,,,,,,
Europe PMC,16.8%,77.1%,0.0%,0.0%,6.8%,0.0%
European Commission,97.2%,100.0%,0.0%,0.0%,0.0%,0.0%
NIHR Funding and Awards,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
UKRI Gateway to Research,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%


In [41]:
combined_authors_dedup.to_csv('/content/drive/MyDrive/Bulk grant data/Research projects awardees.csv', index=False)